<a href="https://colab.research.google.com/github/rajp152k/Machine_Learning/blob/master/Deep%20Learning/Learning%20Pytorch/pytor_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Creating a basic Convolutional Neural Net

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,6,3)
        self.conv2 = nn.Conv2d(6,16,3)
        self.fc1 = nn.Linear(6*6*16,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    
    def forward(self,x):
        print(x.size())
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        print(x.size())
        x = F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        print(x.size())
        x = x.view(-1,self.no_o_flat_feats(x))
        print(x.size())
        x = F.relu(self.fc1(x))
        print(x.size())
        x = F.relu(self.fc2(x))
        print(x.size())
        x = self.fc3(x)
        print(x.size())
        return x
       
    def no_o_flat_feats(self,x):
        size = x.size()[1:]#from 1 cause don't need the batch size (at index 0)
        num_o_feats=1
        for i in size:
            num_o_feats*=i
        
        return num_o_feats

In [23]:
torch.cuda.is_available()

True

In [63]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [0]:
params = list(net.parameters())

In [65]:
params[0].size()# no. of parameters for the Net.conv1 layer

torch.Size([6, 1, 3, 3])

checking on a random 32\*32 input   

In [0]:
input  = torch.randn(1,1,32,32)
# first index = batch size
# second index = no. of channels in one input
# third index = height of an input
# fourth index = width of an input

In [73]:
output = net(input)

torch.Size([1, 1, 32, 32])
torch.Size([1, 6, 15, 15])
torch.Size([1, 16, 6, 6])
torch.Size([1, 576])
torch.Size([1, 120])
torch.Size([1, 84])
torch.Size([1, 10])


In [74]:
print(output)
output.size()

tensor([[-0.0995,  0.0873, -0.0267, -0.1413, -0.1006,  0.0811, -0.1133,  0.0683,
          0.1379, -0.1261]], grad_fn=<AddmmBackward>)


torch.Size([1, 10])

Back propogating

In [0]:
net.zero_grad()

defining a random target tensor for computing a loss

In [76]:
target = torch.randn(1,10)
criterion = nn.MSELoss()#read more on other types of losses
loss = criterion(output,target)
print(loss)

tensor(0.8519, grad_fn=<MseLossBackward>)


In [77]:
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)
loss.backward()
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0108,  0.0080, -0.0015, -0.0040, -0.0056, -0.0048])


updating the weights

In [0]:
LR  = 0.01 # learning rate
for f in net.parameters():
    f.data -= (f.grad.data*LR)

 - will rarely use the basic python code as above
 - effecient optimizers exist

In [0]:
import torch.optim as optim 

optimizer = optim.SGD(net.parameters(),lr=0.01)
#stochastic gradient descent

optimizer.zero_grad() #nullify gradient buffers first

In [83]:
input2 = torch.randn(1,1,32,32)
output = net(input2)
loss2 =criterion(output,target)

loss2.backward()

optimizer.step()

torch.Size([1, 1, 32, 32])
torch.Size([1, 6, 15, 15])
torch.Size([1, 16, 6, 6])
torch.Size([1, 576])
torch.Size([1, 120])
torch.Size([1, 84])
torch.Size([1, 10])
